# Configururation

### Enviornment Configuration

In [1]:
ENV_USE_FRAME_STACK = False
ENV_FRAME_STACK_COUNT = 4
ENV_USE_GRAYSCALE_OBSERVATION = True

### Model Configuration

In [2]:
MODEL_USE_CNN = True
MODEL_DEVICE = "auto" # "auto", "cuda", "cpu", or "mps"

### Traning Configuration

In [4]:
TRAINING_NAME = "CNN-GRAYSCALE-PRI_REPLAY_223"                # Name for the current training (Used for checkpoint-name and log-name)
TRAINING_EVAL_FREQUENCY = 5000                      # How often the agent should be evaluated
TRAINING_EVAL_TIMES = 5                             # How many simulations to perform per evaluation

TRAINING_TOTAL_TIMESTEPS = 600000                   # Total number timesteps before training is complete

TRAINING_LEARNING_RATE = 1e-5                       # Learning rate
TRAINING_LEARNING_STARTS_AT = 10000                 # Timesteps to perform before learing begins

TRAINING_REPLAY_BUFFER_SIZE = 100000                # Number of episode-replays to save in the replay buffer
TRAINING_BATCH_SIZE = 32                            # Number of replays to pick each gradient
TRAINING_TARGET_UPDATE_INTERVAL = 1                 # How often the target network will be updated by online network
TRAINING_GRADIENT_STEPS = -1                        # How many gradient steps shall be performed per training

TRAINING_USE_PRIORITIZED_REPLAY = False             # Wether or not to use Prioritized Replay
TRAINING_PRIORITIZED_REPLAY_EPS = 1e-5              # Epsilon of prioritized replay
TRAINING_PRIORITIZED_REPLAY_BETA_START = 1.0        # Start beta of prioritized replay, beta controls the importance of prioritized replays
TRAINING_PRIORITIZED_REPLAY_BETA_END = 0.1          # End beta-value
TRAINING_PRIORITIZED_REPLAY_BETA_FRACTION = 0.55    # Fraction of total training time for beta-decrease

TRAINING_EPS_START = 0.9                            # Start value for epsilon, epsilon controls exploration rate
TRAINING_EPS_END = 0.0095                           # End value for epsilon
TRAINING_EPS_FRACTION = 0.5                         # Fraction of total training time for epsilon-decrease

TRAINING_GAMMA = 0.99                               # Static gamma, controls how importance uncertain rewards in far future (1.0) and the ones in the near future (0.0)
TRAINING_TAU = 0.975                                 # Static tau, controls how much of the online network shall be copied to the target network at each copy.

# Setup Minedojo Environment

In [5]:
from Environments import SkyRunner, MultithreadGymWrapper

# Multithreaded environment wrapper
env = MultithreadGymWrapper.MultithreadGymWrapper(thread_int=1, env_int=1,
    frame_stack=ENV_USE_FRAME_STACK,
    frames_int=ENV_FRAME_STACK_COUNT,
    use_grayscale=ENV_USE_GRAYSCALE_OBSERVATION
)

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


starting Reloader 0
ThreadID: 0 has received an enviornment from queue. Reset of environement is being prepeared


# Load EVAL-Enviornment

In [7]:
# Evaluation environment
eval_env = SkyRunner.SkyRunnerEnvironment(
    frame_stack=ENV_USE_FRAME_STACK,
    frames_int=ENV_FRAME_STACK_COUNT,
    use_grayscale=ENV_USE_GRAYSCALE_OBSERVATION)
eval_env.reset()

Unable to reset enviornment due to an exception
Traceback (most recent call last):
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/SkyRunner.py", line 92, in reset
    local_obs = self.env.reset()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/Environments/SkyRunnerMissionWrapper.py", line 166, in reset
    self.env.reset()
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/gym/core.py", line 292, in reset
    return self.env.reset(**kwargs)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/minedojo/sim/wrappers/ar_nn/delta_inventory_wrapper.py", line 78, in reset
    observation = self.env.reset(**kwargs)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/gym/core.py", line 319, in reset
    observation = self.env.reset(**kwargs)
  File "/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainA

[INFO:minedojo.tasks] Loaded 1572 Programmatic tasks, 1558 Creative tasks, and 1 special task: "Playthrough". Totally 3131 tasks loaded.


-1

# Begin training

In [7]:
import Train

Train.train(
    env=env,
    eval_env=eval_env,
    name=TRAINING_NAME,
    eval_freq=TRAINING_EVAL_FREQUENCY,                  
    n_eval_episodes=TRAINING_EVAL_TIMES,                           
    total_timesteps=TRAINING_TOTAL_TIMESTEPS,                 
    learning_rate=TRAINING_LEARNING_RATE,                     
    learning_starts=0,
    buffer_size=TRAINING_REPLAY_BUFFER_SIZE,              
    batch_size=TRAINING_BATCH_SIZE,                          
    target_update_interval=TRAINING_TARGET_UPDATE_INTERVAL,              
    gradient_steps=TRAINING_GRADIENT_STEPS,                       
    use_prioritized_replay=TRAINING_USE_PRIORITIZED_REPLAY,           
    prioritized_replay_eps=TRAINING_PRIORITIZED_REPLAY_EPS,           
    prioritized_replay_initial_beta=TRAINING_PRIORITIZED_REPLAY_BETA_START,      
    prioritized_replay_final_beta=TRAINING_PRIORITIZED_REPLAY_BETA_END,        
    prioritized_replay_beta_fraction=TRAINING_PRIORITIZED_REPLAY_BETA_FRACTION,  
    exploration_initial_eps=TRAINING_EPS_START,                          
    exploration_final_eps=TRAINING_EPS_END,                         
    exploration_fraction=TRAINING_EPS_FRACTION ,                       
    gamma=TRAINING_GAMMA,                            
    tau=TRAINING_TAU,
    use_cnn=MODEL_USE_CNN,
    device=MODEL_DEVICE                               
)

/Users/kristian.aars/PycharmProjects/IDATT2502-SkyBrainAI/venv/lib/python3.9/site-packages/stable_baselines3/common/buffers.py:229: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 8.64GB > 5.66GB
  warnings.warn(


Inventory and weather cleared!


KeyboardInterrupt: 

# Preview Trained Model

In [ ]:
from stable_baselines3 import DQN
from CustomBaselines3.DoubleDQN import DoubleDQN

model = DoubleDQN.load("./dDQN-checkpoints/" + TRAINING_NAME + "/final_model.zip")

obs = env.reset()
acc_r = 0
while True:
    act, st = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(act)

    acc_r += reward

    env.render()

    if done:
        obs = env.reset()
        print("Finished with reward %d" % acc_r)
        acc_r = 0

# Shutdown Environments

In [ ]:
env.close()
eval_env.close()